In [1]:
import numpy as np; np.random.seed(0)
import seaborn as sns;
import pandas as pd
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import os
import scipy.stats
import math
from ete3 import Tree
%matplotlib inline  
pylab.rcParams['figure.figsize'] = 15, 15  # our default image size

import json

In [2]:
# load up the allReads file into an array of events
class Event:
    def __init__(self,line,sample):
        self.sample = sample
        
        sp = line.split("\t")
        self.hmid = "_".join([x if x != "UNKNOWN" else "NONE" for x in sp[0].split("_")])
        self.array = int(sp[1])
        self.count = int(sp[2])
        self.proportion = float(sp[3])
        
def load_sample(sample_file, sample):
    header = sample_file.readline().strip("\n").split("\t")
    events = []
    
    for line in sample_file:
        events.append(Event(line,sample))
    return events
    
# the two sample tables to load
old_dir = "/mount/vol10/CRISPR.lineage/nobackup/2016_05_04_Early_embryo_target_6_and_7/data/"
new_dir = "/mount/vol10/CRISPR.lineage/nobackup/2016_05_04_embryo_rerun/data/"

old_samples = open(old_dir + "/crispr_tearsheet.txt")
new_samples = open(new_dir + "/crispr_tearsheet.txt")

old_samples.readline()
new_samples.readline()
sample_names = []
sample_to_events = {}

for line in new_samples:
    sp = line.split("\t")
    filename = new_dir + "/pipeline_output/" + sp[0] + "/" + sp[0] + ".allReadCounts"
    if os.path.exists(filename):
        sample_hmids = load_sample(open(filename),sp[0])
        sample_names.append(sp[0])
        sample_to_events[sp[0]] = sample_hmids


In [11]:
sampling_repeats = 500

# for a sample, pull N HMIDs, ask if we've seen it before, 
# if so the Nth array value gets a 0 or 1.  Average over simulations
def simulate_sample(sample):
    
    # make a linear version of the HMIDs from the count data
    this_sample = sample_to_events[sample]
    hmid_vector = []
    for hmid in this_sample:
        for i in range(0,hmid.count):
            hmid_vector.append(hmid)
    print sample + " of length " + str(len(hmid_vector))
    # now sample from that linear array, and ask if we've seen the HMID already
    unique_count = np.zeros((len(hmid_vector),sampling_repeats))
    
    for repeat in range(0,sampling_repeats):
        seen_HMIDs = {}
        # rand_HMIDs = np.random.randint(0, len(hmid_vector), size=sampled_cells)
        rand_HMIDs = np.random.choice(len(hmid_vector),size=len(hmid_vector),replace=False)
        for index in range(0,len(hmid_vector)):
            hmid = hmid_vector[rand_HMIDs[index]]
            new_val = 1
            if hmid.hmid in seen_HMIDs:
                new_val = 0
            unique_count[index,repeat] = unique_count[index,repeat] + new_val
            seen_HMIDs[hmid.hmid] = True
    
    return unique_count


In [4]:
def results_to_eCDF_vector_and_sd(input_matrix):
    sum_val = 0.0
    num_cols = input_matrix.shape[1]
    
    return_matrix = np.zeros((len(input_matrix),2))
    for i in range(0,len(input_matrix)):
        mean_val = np.mean(input_matrix[i,].flatten())
        return_matrix[i,0] = mean_val + sum_val
        sum_val += mean_val
        
        sub_matrix = np.sum(input_matrix[0:i,],axis=0)
        return_matrix[i,1] = np.std(sub_matrix)
    return return_matrix

In [5]:
def to_output(axis):
    return (str(axis[0,]) + "\t" + str(axis[1,]))

def to_output_file(sample,matrix,output_file):
    concentration = "1X"
    hours = "4.3H"
    
    if "0.3x" in sample:
        concentration = "1/3X"
    if "epi" in sample:
        hours = "9H"
    if "30hr" in sample:
        hours = "30H"
    if "3d" in sample:
        hours = "72H"
    
    for i,row in enumerate(matrix):
        output_str = to_output(row)
        output_file.write(sample + "\t" + concentration + "\t" + hours + "\t" + str(i+1) + "\t" + output_str + "\n")


In [12]:
# main loop
# ------------------------------------------------------------------------------------------
output_file = open("test_simulation_no_replace_version2.txt","w")
output_file.write("sample\tconcentration\thours\tindex\tmean\tstd\n")
def simulate_samples(sample):
    ret = simulate_sample(sample)
    ret2 = results_to_eCDF_vector_and_sd(ret)
    to_output_file(sample,ret2,output_file)

for sample in sample_names:
    if len(sample_to_events[sample]) > 100:
        simulate_samples(sample)
    
    
output_file.close()


Dome_1_1x of length 1262
Dome_3_1x of length 2036
Dome_5_1x of length 861
Dome_8_1x of length 1385
Dome_10_1x of length 2258
Dome_5_0.3x of length 2571
Dome_6_0.3x of length 3028
epi90_2_1x of length 4053
epi90_5_1x of length 4014
epi90_8_1x of length 2901
epi90_9_1x of length 3415
epi90_12_1x of length 6259
epi90_1_0.3x of length 2087
epi90_2_0.3x of length 7686
epi90_3_0.3x of length 3239
epi90_9_0.3x of length 8776
epi90_10_0.3x of length 9480
epi90_12_0.3x of length 6876
30hr_1_1x of length 10697
30hr_2_1x of length 6633
30hr_3_1x of length 24023
30hr_4_1x of length 9006
30hr_5_1x of length 23385
30hr_6_1x of length 15414
30hr_1_0.3x of length 10429
30hr_2_0.3x of length 9430
30hr_3_0.3x of length 14059
30hr_4_0.3x of length 14839
30hr_5_0.3x of length 17878
30hr_6_0.3x of length 15910
3d_1_1x of length 25584
3d_2_1x of length 14691
3d_3_1x of length 17984
3d_4_1x of length 6574
3d_5_1x of length 9541
3d_6_1x of length 9752
3d_1_0.3x of length 21720
3d_2_0.3x of length 25480
3d_3_0